In [1]:
import pathlib
import re

In [2]:
class RunResult:
    Serial="Serial"
    OpenMP="OpenMP"
    MPI="MPI"
    
    def __init__(self, name, millis, procCnt, runType, nodeCnt=None):
        self.name = name
        self.millis = millis
        self.procCnt = procCnt
        self.nodeCnt = nodeCnt
        self.runType = runType
            
        
    def __str__(self):
        return f"""
        {self.name} ({self.runType})\n
        \tDuration [ms]: {self.millis}\n
        \tproc: {self.procCnt}\n
        \tnode: {self.nodeCnt}\n
        """
class BaseParser:
    
    @staticmethod 
    def parseStdout(stdout):
        with open(stdout) as file:
            data = file.read().splitlines()
            
        millis = None
        for i, line in enumerate(data):
            if re.match(f'^Cena', line):
                millis = int(data[i+1].split()[2])
                break
                
        return millis

# Serial

In [3]:
class SerialParser(BaseParser):
    
    @staticmethod 
    def parseFilename(filename):
        return 1, str(filename)
    
    @staticmethod 
    def parse(path):
        procCnt, name = SerialParser.parseFilename(path.name)
        millis = SerialParser.parseStdout(path / 'stdout')
        return RunResult(name, millis, procCnt, RunResult.Serial)
    

pSerial = pathlib.Path('serial')
serialLst = []
for p in pSerial.glob('*'):
    serialLst.append(SerialParser.parse(p))
    
serialKilled = set([x.name+'-p'+str(x.procCnt) for x in serialLst if x.millis is None])
print(f'Killed {serialKilled}')

Killed {'saj6-p1'}


# OpenMP

In [4]:
class OpenMpParser(BaseParser):
    
    @staticmethod 
    def parseFilename(filename):
        sp = str(filename).split('-')
        name = sp[0]
        procCnt = int(sp[1][1:])
        return procCnt, name
    
    @staticmethod 
    def parse(path):
        procCnt, name = OpenMpParser.parseFilename(path.name)
        millis = OpenMpParser.parseStdout(path / 'stdout')
        return RunResult(name, millis, procCnt, RunResult.OpenMP)

## Task parallelism

In [8]:
pTaskPar = pathlib.Path('openmp/task-par/out')
taskParLst = []
for p in pTaskPar.glob('*'):
    taskParLst.append(OpenMpParser.parse(p))
    
taskKilled = set([ x.name+'-p'+str(x.procCnt) for x in taskParLst if x.millis is None])
print(f'Killed {taskKilled}')

Killed {'saj12-p16', 'saj12-p2'}


## Task parallelism [threshold]

In [6]:
pTaskParThr = pathlib.Path('openmp/task-par-threshold/out-t6')
taskParLstThr = []
for p in pTaskParThr.glob('*'):
    taskParLstThr.append(OpenMpParser.parse(p))
    
taskKilled = set([ x.name+'-p'+str(x.procCnt) for x in taskParLstThr if x.millis is None])
print(f'Killed {taskKilled}')

Killed {'saj12-p6', 'saj12-p4', 'saj12-p10', 'saj12-p16', 'saj12-p2', 'saj12-p8'}


## Data parallelism 

In [7]:
pDataPar = pathlib.Path('openmp/data-par/out0')
dataParLst = []
for p in pDataPar.glob('*'):
    dataParLst.append(OpenMpParser.parse(p))

dataKilled = set([x.name+'-p'+str(x.procCnt) for x in dataParLst if x.millis is None])
print(f'Killed {dataKilled}')

Killed set()


# MPI